In [8]:
!pip install transformers torch


In [9]:
# ===== MUST BE AT THE TOP =====
import os
import warnings
from transformers import pipeline, logging

# 1. Disable all Python warnings
warnings.filterwarnings("ignore")

# 2. Disable Hugging Face / Transformers logs
logging.set_verbosity_error()

# 3. Disable Hugging Face progress bars (config, tokenizer, model downloads)
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"

# ===== YOUR ACTUAL CODE =====
prompt = "The future of Artificial Intelligence is"

models = {
    "BERT": "bert-base-uncased",
    "RoBERTa": "roberta-base",
    "BART": "facebook/bart-base"
}

for name, model in models.items():
    print(f"\nModel: {name}")
    try:
        generator = pipeline(
            "text-generation",
            model=model,
            device=-1  # force CPU
        )
        output = generator(prompt, max_new_tokens=15)
        print(output[0]["generated_text"])
    except Exception:
        print("Generation failed")



Model: BERT
The future of Artificial Intelligence is...............

Model: RoBERTa
The future of Artificial Intelligence is

Model: BART
The future of Artificial Intelligence isinvolved TreesGUamsung272involved Trees NavimmerDEBUGDEBUGDEBUG Thin Dum


In [10]:
from transformers import pipeline

sentences = {
    "BERT": ("bert-base-uncased", "The goal of Generative AI is to [MASK] new content."),
    "RoBERTa": ("roberta-base", "The goal of Generative AI is to <mask> new content."),
    "BART": ("facebook/bart-base", "The goal of Generative AI is to <mask> new content.")
}

for name, (model, text) in sentences.items():
    print(f"\nModel: {name}")
    try:
        masker = pipeline("fill-mask", model=model)
        results = masker(text)
        for r in results[:3]:
            print(r["token_str"], "-", round(r["score"], 4))
    except Exception as e:
        print("Error:", e)



Model: BERT
create - 0.5397
generate - 0.1558
produce - 0.0541

Model: RoBERTa
 generate - 0.3711
 create - 0.3677
 discover - 0.0835

Model: BART
 create - 0.0746
 help - 0.0657
 provide - 0.0609


In [11]:
from transformers import pipeline

context = "Generative AI poses significant risks such as hallucinations, bias, and deepfakes."
question = "What are the risks?"

models = {
    "BERT": "bert-base-uncased",
    "RoBERTa": "roberta-base",
    "BART": "facebook/bart-base"
}

for name, model in models.items():
    print(f"\nModel: {name}")
    try:
        qa = pipeline("question-answering", model=model)
        result = qa(question=question, context=context)
        print("Answer:", result["answer"])
        print("Score:", round(result["score"], 4))
    except Exception as e:
        print("Error:", e)



Model: BERT
Answer: hallucinations,
Score: 0.0097

Model: RoBERTa
Answer: deepfakes
Score: 0.0121

Model: BART
Answer: significant risks such as hallucinations, bias,
Score: 0.0483


| Task | Model | Classification (Success/Failure) | Observation (What actually happened?) | Why did this happen? (Architectural Reason) |
|------|-------|----------------------------------|--------------------------------------|-------------------------------------------|
| **Generation** | BERT | Failure | The model produced repetitive symbols instead of meaningful text. | BERT is an encoder-only model and is not trained to predict the next word. |
|  | RoBERTa | Failure | The model failed to generate any continuation beyond the prompt. | RoBERTa is also an encoder-only model without autoregressive generation capability. |
|  | BART | Partial Success | The model generated text, but the output was incoherent and nonsensical. | BART has an encoder-decoder architecture, but the base model is not fine-tuned for open-ended text generation. |
| **Fill-Mask** | BERT | Success | The model correctly predicted words such as “create” and “generate” with high confidence. | BERT is trained using Masked Language Modeling (MLM). |
|  | RoBERTa | Success | The model accurately predicted suitable missing words with strong confidence. | RoBERTa is optimized for MLM with improved training strategy and data. |
|  | BART | Partial Success | The model predicted reasonable words but with lower confidence. | BART is trained for sequence-to-sequence denoising, not primarily for MLM tasks. |
| **QA** | BERT | Partial Failure | The model extracted a long phrase from the context but with very low confidence. | The base BERT model is not fine-tuned for Question Answering tasks such as SQuAD. |
|  | RoBERTa | Failure | The model returned an incomplete one-word answer with very low confidence. | RoBERTa base lacks QA fine-tuning and an effective span-prediction head. |
|  | BART | Partial Failure | The model extracted a slightly relevant but incomplete answer with low confidence. | Although BART supports seq2seq tasks, it is not trained for extractive QA in its base form. |
